In [1]:
import os
from glob import glob
import sys
sys.path.insert(0, '../core')

from load.load_data import dirtyMNISTDataset

In [5]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from skimage import io

In [15]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
dataset = dirtyMNISTDataset(mode='train')
dataloader = DataLoader(dataset=dataset, batch_size=)

In [22]:
class VanillaCNN(nn.Module):
    def __init__(self, input_size=(256, 256), hidden_size=(32, 64), num_classes=26,  init_weight='he', init_bias='zero'):
        super(VanillaCNN, self).__init__()
        self.init_weight = init_weight
        self.init_bias = init_bias

        layer_list = []
        prev_channel = input_size[0]

        for idx in range(len(hidden_size)):
            layer_list.append(nn.Conv2d(prev_channel, hidden_size[idx], kernel_size=3, stride=1, padding=1))
            layer_list.append(nn.BatchNorm2d(hidden_size[idx]))
            layer_list.append(nn.ReLU(True))
            layer_list.append(nn.MaxPool2d(kernel_size=2, stride=2))
            prev_channel = hidden_size[idx]
        
        layer_list.append(nn.Flatten())
        feature_size = int(input_size[1] / 2**len(hidden_size))
        layer_list.append(nn.Linear(in_features=(feature_size**2)*prev_channel, out_features=num_classes))

        self.net = nn.Sequential(*layer_list)
        self.init_params()

    def init_params(self):
        init_weight_method = {'he': nn.init.kaiming_normal_, 'xavier': nn.init.xavier_normal_}
        assert self.init_weight in init_weight_method.keys(), f'Select the weight initialization method in {list(init_weight_method.keys())}'

        init_bias_method = {'zero': nn.init.zeros_, 'uniform': nn.init.uniform_}
        assert self.init_bias in init_bias_method.keys(), f'Select the bias initialization method in {list(init_bias_method.keys())}'

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init_weight_method[self.init_weight](m.weight)
                init_bias_method[self.init_bias](m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, X):
        return self.net.forward(X)


In [26]:
from torchsummary import summary

In [27]:
summary(model, input_size=(256, 256))

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [32, 256, 3, 3], but got 3-dimensional input of size [2, 256, 256] instead

In [23]:
model = VanillaCNN().to(device)

In [25]:
model.forward(X=torch.Tensor(dataset[0]['image']))

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [32, 256, 3, 3], but got 2-dimensional input of size [256, 256] instead

In [19]:
dataset[0]

{'image': array([[ 26, 196,   1, ...,  74, 154,  20],
        [176, 223,  15, ...,  61, 104, 123],
        [114, 131, 239, ...,  79, 199,  27],
        ...,
        [198,   1, 159, ..., 254, 106, 212],
        [147, 236, 140, ..., 215,  43, 253],
        [192,  45, 154, ..., 170, 157, 233]], dtype=uint8),
 'label': array([1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
        0, 1, 1, 1])}

In [11]:
img_path = glob('../data/dirty_mnist/train/*')[0]

In [13]:
io.imread(img_path).shape

(256, 256)